In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install -U feature-engine

In [ ]:
import catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.samplers import TPESampler
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split

In [ ]:
#Optuna objective function
def objective(trial):
    params = {
        #"iterations": 3000,    # OPtuna example not specify iteration
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostRegressor(**params, silent=True,random_seed=51)
    model.fit(train_pool,verbose=0,eval_set=val_pool)
    y_pred = model.predict(val_pool)
    #rmse = mean_squared_error(y_val, predictions, squared=False)
    return eval_metric(val_pool.get_label(),y_pred,'RMSE')

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

In [ ]:
#df_train['StartYM']=pd.to_datetime(df_train['Policy Start Date']).dt.strftime('%Y-%m')
df_train['StartY']=pd.to_datetime(df_train['Policy Start Date']).dt.strftime('%Y')
df_test['StartY']=pd.to_datetime(df_test['Policy Start Date']).dt.strftime('%Y')

In [ ]:
#df_train['StartYM'].value_counts()

In [ ]:
df_train['StartY'].value_counts().reset_index().sort_values(by=['StartY'],ascending=False)

In [ ]:
df_train[df_train['Premium Amount']<120].groupby('StartY')['Premium Amount'].describe()

In [ ]:
df_test['Annual Income'].describe()

In [ ]:
df_train[df_train['Annual Income']<200]['Credit Score'].value_counts()

In [ ]:
df_test[df_test['Annual Income']<200]['Annual Income'].value_counts()

In [ ]:
#only Age, Anuual Income,Credit Score are number. The rest are category
numeric_columns=df_train.select_dtypes(include=np.number).columns
numeric_columns

In [ ]:
object_columns=df_train.select_dtypes(include=object).columns
object_columns

In [ ]:
#transform target
df_train['Label']=np.log1p(df_train['Premium Amount'])
df_train['Label'].describe()

In [ ]:
#Check Na credit score and premium amount
# There a lot of missing score with a lot of premium no different with having creditscor
df_train[df_train['Credit Score'].notna()].groupby('Premium Amount').size().reset_index(name='count').assign(total=lambda x:x['Premium Amount']*x['count']).groupby('count')['total'].sum()

In [ ]:
#from feature_engine.transformation import YeoJohnsonTransformer

#tf = YeoJohnsonTransformer(variables = ['Annual Income', 'Label'])

#tf.fit(df_train.dropna())
#tf.lambda_dict_

In [ ]:
#df_train_tf=tf.transform(df_train.dropna())

In [ ]:
df_train[df_train['Premium Amount']<120].loc[:,['Annual Income','Credit Score']].sort_values(ascending=True,by=['Annual Income']).head()

In [ ]:
df_train[df_train['Premium Amount']<120]['Credit Score'].describe()

In [ ]:
df_train['Credit Score'].describe()

In [ ]:
df_train.info()

In [ ]:
# change fillna from mean to mode to get less effect from outlier
# Ana=df_train['Age'].mode()[0]
# Aina=df_train['Annual Income'].mode()[0]
# Ndna=0
# Hsna=df_train['Health Score'].mode()[0]
# Pcna=0
# Vana=0 #just impute with 0
# Csna=df_train['Credit Score'].mode()[0]
# Idna=0

In [ ]:
# df_train['Age'].fillna(Ana,inplace=True)
# df_train['Annual Income'].fillna(Aina,inplace=True)
# df_train['Number of Dependents'].fillna(Ndna,inplace=True)
# df_train['Health Score'].fillna(Hsna,inplace=True)
# df_train['Previous Claims'].fillna(Pcna,inplace=True)
# df_train['Vehicle Age'].fillna(Vana,inplace=True)
# df_train['Credit Score'].fillna(Csna,inplace=True)
# df_train['Insurance Duration'].fillna(Idna,inplace=True)


#Ana=df_test['Age'].mean()
#Aina=df_test['Annual Income'].mean()
#Ndna=0
#Hsna=df_test['Health Score'].mean()
#Pcna=0
#Vana=0 #just impute with 0
#Csna=df_test['Credit Score'].mean()
#Idna=0

# df_test['Age'].fillna(Ana,inplace=True)
# df_test['Annual Income'].fillna(Aina,inplace=True)
# df_test['Number of Dependents'].fillna(Ndna,inplace=True)
# df_test['Health Score'].fillna(Hsna,inplace=True)
# df_test['Previous Claims'].fillna(Pcna,inplace=True)
# df_test['Vehicle Age'].fillna(Vana,inplace=True)
# df_test['Credit Score'].fillna(Csna,inplace=True)
# df_test['Insurance Duration'].fillna(Idna,inplace=True)



In [ ]:
import numpy as np
from catboost import CatBoostRegressor
# Initialize data

train_data = [[1., 4., 5., np.nan],
              [4., 5., 6., 7.],
              [30., 40., 50., 60.]]

eval_data = [[2., 4., 6., 8.],
             [1., 4., 50., 60.]]

train_labels = [10., 20., 30.]
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(train_data, train_labels)
# Get predictions
preds = model.predict(eval_data)

In [ ]:
df_train[df_train['Age'].isna()].index

In [ ]:
df_train['Occupation'].fillna('Unknown',inplace=True)
df_train['Marital Status'].fillna('Unknown',inplace=True)
df_train['Customer Feedback'].fillna('Unknown',inplace=True)

df_test['Occupation'].fillna('Unknown',inplace=True)
df_test['Marital Status'].fillna('Unknown',inplace=True)
df_test['Customer Feedback'].fillna('Unknown',inplace=True)


In [ ]:
#cat_features=numeric_columns.drop(['id','Age','Annual Income','Credit Score','Health Score','Premium Amount']).union(object_columns)
cat_features=object_columns
cat_features.to_list()


In [ ]:
#change cat columns datatype to astype int
df_train[cat_features.intersection(numeric_columns)]=df_train[cat_features.intersection(numeric_columns)].astype('Int64')

#text_features = cat_features.remove('Vehicle Age')

In [ ]:
#change cat columns datatype to int astype
df_test[cat_features.intersection(numeric_columns)]=df_test[cat_features.intersection(numeric_columns)].astype('Int64')

#text_features = cat_features.remove('Vehicle Age')

In [ ]:
#custom loss function but will not be able to optimization with it so better transform target instead
#def rmsle_loss(y_true, y_pred):
#    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))
#Also use exp to transform the prediction
#predictions = np.expm1(model.predict(X_test))

In [ ]:
#df_train.drop(['id','Annual Income','Customer Feedback','Exercise Frequency','Premium Amount','group Age','Label'],axis=1).columns

In [ ]:
#df_train2 = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
#df_train2=df_train2.dropna()


In [ ]:
#nonadata = pd.concat([df_train2['Credit Score'],df_train2['Health Score']],axis=1)
#nonadata.head()

In [ ]:
#nonadata.describe()
#from sklearn.preprocessing import StandardScaler, RobustScaler

#standard_scaler=StandardScaler()
#scalenonadata=standard_scaler.fit_transform(nonadata)

#robust_scaler=RobustScaler()
#scalenonadata=robust_scaler.fit_transform(nonadata)

In [ ]:
#nonadata=pd.DataFrame(scalenonadata,columns=['Credit Score','Health Score'])

In [ ]:
#df_train2['Label'] = np.log1p(df_train2['Premium Amount'])

In [ ]:
data = df_train.drop(['id','Premium Amount','Policy Start Date','Label'],axis=1) 
cat_features=cat_features.intersection(data.columns)
cat_features=cat_features.difference(['Credit Score','Health Score','Annual Income'])
#text_features=text_features.intersection(data.columns)

In [ ]:
cat_features

In [ ]:
data.columns

In [ ]:
numeric_columns

In [ ]:
df_train[cat_features]=df_train[cat_features].astype('category')
df_test[cat_features]=df_test[cat_features].astype('category')

In [ ]:
df_train.info()

In [ ]:
from catboost import *

X_train,x_val,Y_train,y_val = train_test_split(df_train.drop(['id','Premium Amount','Policy Start Date','Label'],axis=1),df_train['Label'],test_size=0.2,random_state=11)
train_pool=Pool(data=X_train,
                label=Y_train,
                cat_features=cat_features.to_list())
val_pool=Pool(data=x_val,
               label=y_val,
               cat_features=cat_features.to_list())
X_test = df_test.drop(['id','Policy Start Date'],axis=1)


In [ ]:
def calc_test_quality(train_pool, val_pool, **kwargs):
    model = CatBoostRegressor(**kwargs, random_seed=11)
    model.fit(train_pool, verbose=0, eval_set=val_pool)
    y_pred = model.predict(val_pool)
    return eval_metric(val_pool.get_label(), y_pred, 'RMSE'), model

In [ ]:
from optuna.samplers import TPESampler

sampler = TPESampler(seed=123)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=20)

In [ ]:
best_params={'learning_rate': 0.07717313793943478,
            'depth': 10, 
            'l2_leaf_reg': 6.673721788075379,
            'subsample': 0.6913428855236023, 
            'colsample_bylevel': 0.8273248556818064,
            'min_data_in_leaf': 8}
#Best is trial 17 with value: 1.0496756348707192

metriclist, model = calc_test_quality(train_pool,val_pool,**best_params)
model.save_model('optuna_tune_model')

In [ ]:
predicts = model.predict(X_test)

In [ ]:
predicts_original_scale = np.expm1(predicts)

In [ ]:
#round int
temp=pd.concat([df_test['id'],pd.DataFrame(np.rint(predicts_original_scale),columns=['Premium Amount'])],axis=1).reset_index(drop=True)

In [ ]:
#not round int
temp2=pd.concat([df_test['id'],pd.DataFrame(predicts_original_scale.astype(int),columns=['Premium Amount'])],axis=1).reset_index(drop=True)

In [ ]:
temp.to_csv('submissionX.csv',index=False)

In [ ]:
temp2.to_csv('submission_not_round.csv',index=False)

In [ ]:
!zip sub.zip submissionX.csv